In [1]:
using SplittingOrderConditions

In [244]:
p,s = 3,3
@time begin
    eqs = generate_equations(1,s, with_brackets=false)
    for q=2:p
       merge!(eqs,generate_equations(q,s,  with_brackets=false)) 
    end
end    
eqs

  0.000239 seconds (1.54 k allocations: 91.813 KB)


Dict{Array{Int64,1},AbstractString} with 5 entries:
  [0,1]   => "+2*b1*a2+2*b1*a3+2*b2*a3-1"
  [0]     => "+1*a1+1*a2+1*a3-1"
  [0,0,1] => "+3*b1*a2^2+6*b1*a2*a3+3*b1*a3^2+3*b2*a3^2-1"
  [0,1,1] => "+3*b1^2*a2+3*b1^2*a3+6*b1*b2*a3+3*b2^2*a3-1"
  [1]     => "+1*b1+1*b2+1*b3-1"

In [3]:
vars = ["a1", "a2", "a3", "b1", "b2", "b3"]
funs = Dict{Array{Int64,1},Function}()
for lw in keys(eqs)
    for k =1:length(vars)
        eqs[lw] = replace(eqs[lw], vars[k], string("x(",k,")"))        
    end
    funs[lw] = SplittingOrderConditions.compile_fg(eqs[lw], length(vars))
end    
funs=collect(values(funs))

# of attemps to call form = 3457
# of attemps to call form = 97
# of attemps to call form = 919
# of attemps to call form = 368
# of attemps to call form = 489


5-element Array{Function,1}:
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)

In [4]:
eqs_lem = generate_equations(p+1,s, with_brackets=false)
for lw in keys(eqs_lem)
    for k =1:length(vars)
        eqs_lem[lw] = replace(eqs_lem[lw], vars[k], string("x(",k,")"))        
    end    
end
lem_eq = join([string("(",eq,")^2") for eq in values(eqs_lem)], :+)
lem = SplittingOrderConditions.compile_fg(lem_eq, length(vars))

# of attemps to call form = 86


(anonymous function)

In [5]:
using NLopt

In [16]:
#opt = Opt(:LD_MMA, 2*s)
opt = Opt(:GN_ISRES, 2*s)
lower_bounds!(opt, -2.0*ones(2*s))
upper_bounds!(opt, +2.0*ones(2*s))
xtol_rel!(opt,1e-4)

In [17]:
min_objective!(opt, lem)

In [19]:
for f in funs
    equality_constraint!(opt, f, 1e-8)
end

In [21]:
(minf,minx,ret) = optimize(opt, rand(2s))

LoadError: LoadError: InterruptException:
while loading In[21], in expression starting on line 1

In [293]:
function differentiate(eq, var)
    eq1=""
    terms = split(eq,['+','-'], keep=false)
    signs = [m.match for m in collect(eachmatch(r"[\+-]",eq))]
    for t in terms
        s = split(t,'*')        
        c = s[1]
        facs = Dict([ contains(f,"^") ? (split(f,"^")...) : (f, "1")   for f in s[2:end]])
        if var in keys(facs)
            e = parse(facs[var])
            c = string(parse(c)*e)
            if e==1
                delete!(facs, var)
            else    
                facs[var] = string(e-1)
            end
            eq1 = string(eq1, "+", join(vcat(c,[e=="1" ? v : string(v,"^",e ) for (v,e) in facs]), '*'))
        end       
    end
    if eq1==""
        eq1="0"
    end
    eq1
end

differentiate (generic function with 1 method)

In [331]:
p,s = 3,3
@time begin
    eqs = generate_equations(1,s, with_brackets=false)
    for q=2:p
       merge!(eqs,generate_equations(q,s,  with_brackets=false)) 
    end
end    
eqs=collect(values(eqs))

  0.000238 seconds (1.54 k allocations: 91.813 KB)


5-element Array{AbstractString,1}:
 "+2*b1*a2+2*b1*a3+2*b2*a3-1"                 
 "+1*a1+1*a2+1*a3-1"                          
 "+3*b1*a2^2+6*b1*a2*a3+3*b1*a3^2+3*b2*a3^2-1"
 "+3*b1^2*a2+3*b1^2*a3+6*b1*b2*a3+3*b2^2*a3-1"
 "+1*b1+1*b2+1*b3-1"                          

In [332]:
vars = ["a1", "a2", "a3", "b1", "b2", "b3"]

6-element Array{ASCIIString,1}:
 "a1"
 "a2"
 "a3"
 "b1"
 "b2"
 "b3"

In [333]:
eqs_lem = collect(values(generate_equations(p+1,s, with_brackets=false)))
lem_eq = join([string("(",eq,")^2") for eq in eqs_lem], :+)

"(+4*b1^3*a2+4*b1^3*a3+12*b1^2*b2*a3+12*b1*b2^2*a3+4*b2^3*a3-1)^2+(+4*b1*a2^3+12*b1*a2^2*a3+12*b1*a2*a3^2+4*b1*a3^3+4*b2*a3^3-1)^2+(+6*b1^2*a2^2+12*b1^2*a2*a3+6*b1^2*a3^2+12*b1*b2*a3^2+6*b2^2*a3^2-1)^2"

In [334]:
lagrange_eqs = vcat(eqs,
[join(vcat([string("2*(",eq,")*(",differentiate(eq,var),")") for eq in eqs_lem],
    [string("l", j, "*(", differentiate(eqs[j],var),")") for j=1:length(eqs)]),:+) for var in vars]
)
lagrange_eqs = [string(eq,";") for eq in lagrange_eqs]

11-element Array{ByteString,1}:
 "+2*b1*a2+2*b1*a3+2*b2*a3-1;"                                                                                                                                                                                                                                                                                                                                                                                                              
 "+1*a1+1*a2+1*a3-1;"                                                                                                                                                                                                                                                                                                                                                                                                                       
 "+3*b1*a2^2+6*b1*a2*a3+3*b1*a3^2+3*b2*a3^2-1;"                                                               

In [335]:
vars

6-element Array{ASCIIString,1}:
 "a1"
 "a2"
 "a3"
 "b1"
 "b2"
 "b3"

In [336]:
using PyCall
@pyimport phcpy
@pyimport phcpy.solver as solver
@pyimport phcpy.solutions as solutions

In [337]:
@time sols = solver.solve(lagrange_eqs, tasks=4)


ROOT COUNTS :

total degree : 691488
4-homogeneous Bezout number : 392
  with partition : { b1 b2 }{ a2 a3 }{ a1 b3 }{ l1 l2 l3 l4 l5 }
general linear-product Bezout number : 4186
  based on the set structure :
     { b1 b2 }{ a2 a3 }
     { a2 a3 a1 }
     { b1 b2 }{ a2 a3 }{ a2 a3 }
     { b1 b2 }{ b1 b2 }{ a2 a3 }
     { b1 b2 b3 }
     { b1 b2 l1 l2 l3 l4 l5 }{ b1 a2 a3 }{ b1 a2 a3 b2 }{ a2 a3 b2 }
     { b1 l2 l5 }{ b1 b2 l1 l3 }{ b1 a2 a3 l4 }{ b1 a2 a3 b2 }{ b1 a2 a3 }{ b1 a2 a3 b2 }{ a2 a3 b2 }
     { b1 b2 l2 l5 }{ b1 b2 l1 l3 }{ b1 a2 a3 l4 }{ b1 a2 a3 b2 }{ b1 a2 a3 b2 }{ b1 a2 a3 b2 }{ a2 a3 b2 }
     { b1 b2 l1 l2 l3 l5 }{ b1 a2 a3 }{ b1 a2 a3 b2 l4 }{ b1 a2 a3 b2 }{ b1 a2 a3 b2 }{ a2 a3 b2 }{ a2 a3 b2 }
     { b1 b2 l1 l2 l3 l5 }{ b1 a2 a3 }{ b1 a3 b2 l4 }{ b1 a2 a3 b2 }{ b1 a3 b2 }{ a2 a3 b2 }{ a3 b2 }
     { b1 b2 l1 l2 l3 l4 l5 }{ b1 a2 a3 }{ b1 a2 a3 b2 }{ a2 a3 b2 }
mixed volume : 68
stable mixed volume : 68
  7.812857 seconds (129 allocations: 12.188 KB)


9-element Array{Any,1}:
 "t :  1.00000000000000E+00   2.58714722893939E+02\nm : 1\nthe solution for t :\n b1 :  6.54843862488956E-01  -1.93136042802593E-22\n a2 :  7.82907848367289E-01   6.69722835689236E-22\n a3 : -6.71975151492813E-02  -5.42143203016265E-22\n b2 : -4.66110702199038E-01   2.94674484765691E-21\n a1 :  2.84289666781992E-01  -1.27579612193539E-22\n b3 :  8.11266839710083E-01  -2.75360879478269E-21\n l1 :  9.88064647755892E-01   3.97779436387118E-21\n l2 :  4.31308743454427E+00  -4.27889237343615E-01\n l3 : -8.49072722555352E-01  -1.27944662496848E-21\n l4 : -5.21701538705868E-01  -3.30956206339061E-23\n l5 : -4.22190555205767E+00   4.27889237343615E-01\n== err :  2.792E-14 = rco :  0.000E+00 = res :  8.506E-16 ="
 "t :  1.00000000000000E+00   1.68846428289058E+02\nm : 1\nthe solution for t :\n b1 :  3.87747410753696E-01  -1.00071120693573E-01\n a2 :  4.10612900985895E-01   2.06043441934941E-01\n a3 :  3.87747410753697E-01  -1.00071120693576E-01\n b2 :  4.10612900985895E-

In [338]:
eqs_lem = collect(values(generate_equations(p+1,s)))
lem_eq = join([string("(",eq,")^2") for eq in eqs_lem], :+)
@time lem = eval(parse(string( "(a,b)->sqrt( begin ", lem_eq, " end )")))

  0.004300 seconds (363 allocations: 23.255 KB)


(anonymous function)

In [339]:
vars = ["a1", "a2", "a3","b1", "b2", "b3"]
lem_min_real = 1e10
idx_min_real = -1
lem_min_real_1 = 1e10
idx_min_real_1 = -1
idx = 0
a=[0]
b=[0]
for sol in sols
    soldic = solutions.strsol2dict(sol)
    coeffs = Complex128[soldic[var] for var in vars]
    idx += 1
    i = maximum(abs(imag(coeffs)))
    if i<1e-12
        coeffs = Float64[real(x) for x in coeffs]
    end    
    a = coeffs[1:3]
    b = coeffs[4:6]
    n = lem(a,b)
    print("i=", idx, " lem=", n)
    if i<1e-12 
        print(" R")
        if n< lem_min_real
            lem_min_real = n
            idx_min_real = idx
            print(" MIN")
        end
    end
    println()
end    

i=1 lem=0.3330941503269302 R MIN
i=2 lem=0.15420074248480004 - 0.20642265607595806im
i=3 lem=0.2504481506653464 R MIN
i=4 lem=0.38366803502378877 + 0.00016890347685912367im
i=5 lem=0.38366803502378855 - 0.00016890347685963016im
i=6 lem=0.24787675359346228 R MIN
i=7 lem=19.893256498163133 R
i=8 lem=0.33309415032693107 R
i=9 lem=7.667496523556165 R


In [340]:
println("MIN_REAL (idx=", idx_min_real,", lem=", lem_min_real, "):")  
soldic = solutions.strsol2dict(sols[idx_min_real])
coeffs = Float64[real(soldic[var]) for var in vars]
a = coeffs[1:3]
b = coeffs[4:6]
println(a) 
println(b)

MIN_REAL (idx=6, lem=0.24787675359346228):
[0.268330095781765,-0.187991618799098,0.919661523017333]
[0.919661523017466,-0.187991618799221,0.268330095781755]
